# Idea del Proyecto: Predicción del Éxito de una Canción
Paso 1: Recopilación de Datos

a) Datos de Spotify API:

Utiliza la API de Spotify para recopilar datos sobre las características de las canciones. Puedes obtener información sobre el tempo, la energía, la bailabilidad, la popularidad, etc.

b) Scraping Web:

Realiza scraping en una página como Billboard para obtener datos sobre el rendimiento de las canciones en las listas de éxitos. Esto te dará una idea de qué canciones fueron exitosas y cuáles no. Puedes buscar la posición en las listas de las canciones, la cantidad de semanas que estuvieron en las listas, etc.


Paso 2: Procesamiento y Análisis de Datos
Combina los datos de la API de Spotify con los datos de scraping para crear un conjunto de datos completo.
Realiza un análisis exploratorio de datos para comprender mejor las características de las canciones exitosas.
Limpia y prepara los datos para el modelado.

Paso 3: Modelado y Predicción
a) Algoritmos de Clasificación o Predicción:

Utiliza algoritmos de clasificación o regresión para predecir el éxito de una canción.
Algunos algoritmos que podrías probar incluyen:
Regresión Logística
Árboles de Decisión
Bosques Aleatorios
Gradient Boosting
Redes Neuronales
b) Evaluación del Modelo:

Divide tus datos en conjuntos de entrenamiento y prueba.
Entrena varios modelos y evalúa su rendimiento utilizando métricas como precisión, exhaustividad, F1-score, etc.



# Implementación:
1. Recopilación de datos:
Utiliza la API de Spotify para recopilar datos sobre las características de las canciones. Puedes usar la biblioteca spotipy para Python.
Realiza scraping en la página de Billboard para obtener datos sobre el rendimiento de las canciones en las listas de éxitos.

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Credenciales de la API de Spotify
client_id = '3fc2f3c071d8480b96aa379242a07502'
client_secret = 'b838c1a11677441eb4a633bda27b9873'


# Inicialización de la API de Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Obtener datos de una canción (ejemplo)
def get_track_features(track_id):
    track_info = sp.track(track_id)
    features = sp.audio_features(track_id)[0]
    return {
        'track_id': track_info['id'],
        'track_name': track_info['name'],
        'artist_name': track_info['artists'][0]['name'],
        'popularity': track_info['popularity'],
        'danceability': features['danceability'],
        'energy': features['energy'],
        'key': features['key'],
        'loudness': features['loudness'],
        'mode': features['mode'],
        'speechiness': features['speechiness'],
        'acousticness': features['acousticness'],
        'instrumentalness': features['instrumentalness'],
        'liveness': features['liveness'],
        'valence': features['valence'],
        'tempo': features['tempo']
    }

# Lista de IDs de canciones de ejemplo
track_ids = ['1G391cbiT3v3Cywg8T7DM1']

# Obtener datos de ejemplo
tracks_data = [get_track_features(track_id) for track_id in track_ids]

# Convertir a DataFrame de pandas
df_spotify = pd.DataFrame(tracks_data)
print(df_spotify)


                 track_id   track_name            artist_name  popularity  \
0  1G391cbiT3v3Cywg8T7DM1  Scar Tissue  Red Hot Chili Peppers          83   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.595   0.717    0    -4.803     1       0.0295        0.0779   

   instrumentalness  liveness  valence   tempo  
0           0.00274     0.108    0.547  88.969  


2. Procesamiento y Análisis de Datos:

In [10]:
import requests
from bs4 import BeautifulSoup

def get_billboard_rank(artist, song):
    # Formatear el nombre del artista y la canción para la URL
    formatted_artist = artist.replace(' ', '-')
    formatted_song = song.replace(' ', '-')

    # URL de Billboard
    url = f'https://www.billboard.com/music/{formatted_artist}/chart-history/{formatted_song}'

    # Realizar la solicitud HTTP
    response = requests.get(url)
    if response.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar la posición en la lista de Billboard
        rank_element = soup.find('div', class_='chart-list-item__rank')
        if rank_element:
            rank = rank_element.get_text().strip()
            return rank
        else:
            return 'No se encontraron datos en Billboard para esta canción.'
    else:
        return 'Error al conectar con Billboard.'

# Obtener la posición en la lista de Billboard para la canción "Scar Tissue" de Red Hot Chili 
# https://www.billboard.com/music/red-hot-chili-peppers/chart-history/scar-tissue
artist = "red hot chili peppers"
song = "scar tissue"
billboard_rank = get_billboard_rank(artist, song)
print(f'Posición en la lista de Billboard para "{song}" de {artist}: {billboard_rank}')


# # Combinar datos de Spotify con datos de scraping
# # (En este ejemplo, vamos a usar datos simulados)

# # Simulación de datos de scraping (posición en las listas de Billboard)
# billboard_data = {
#     'track_id': ['1G391cbiT3v3Cywg8T7DM1'],
#     'billboard_rank': [1]
# }

# df_billboard = pd.DataFrame(billboard_data)

# # Combinar los dos dataframes
# df_combined = pd.merge(df_spotify, df_billboard, on='track_id', how='left')

# # Análisis exploratorio de datos
# print(df_combined.describe())

# # Limpieza y preparación de datos (tratamiento de valores nulos, etc.)
# # En este caso, no hay valores nulos para tratar.


Posición en la lista de Billboard para "scar tissue" de red hot chili peppers: Error al conectar con Billboard.


3. Modelado y Predicción:

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dividir los datos en conjunto de entrenamiento y prueba
X = df_combined[['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                 'instrumentalness', 'liveness', 'valence', 'tempo']]
y = df_combined['billboard_rank'].apply(lambda x: 1 if x <= 10 else 0)  # Éxito: Posición en el top 10 de Billboard

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Bosques Aleatorios
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicción
y_pred = rf_model.predict(X_test)

# Evaluación del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.0

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:15